In [178]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Date, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [179]:
from datetime import date
import pandas as pd
import os

In [180]:
import json

with open('../../data/games.json') as json_file:
    data = json.load(json_file)

In [181]:
game = data[0]

In [182]:
game['id']

174430

In [183]:
game['name']

'Gloomhaven'

In [184]:
len(game['image'])

89

## Stats

In [185]:
game['stats']

{'usersrated': 34829,
 'average': 8.83196,
 'bayesaverage': 8.576,
 'stddev': 1.60752,
 'median': 0.0,
 'owned': 55995,
 'trading': 346,
 'wanting': 1417,
 'wishing': 14645,
 'numcomments': 6545,
 'numweights': 1494,
 'averageweight': 3.8253,
 'ranks': [{'id': '1',
   'name': 'boardgame',
   'friendlyname': 'Board Game Rank',
   'value': 1},
  {'id': '5496',
   'name': 'thematic',
   'friendlyname': 'Thematic Rank',
   'value': 1},
  {'id': '5497',
   'name': 'strategygames',
   'friendlyname': 'Strategy Game Rank',
   'value': 1}]}

In [186]:
game['designers']

['Isaac Childres']

In [187]:
maximum = 0
for game in data:
    if game['description']:
        if len(game['description']) > maximum:
            maximum = len(game['description'])

In [188]:
maximum = 0
for game in data:
    if game['image']:
        if len(game['image']) > maximum:
            maximum = len(game['image'])

In [189]:
name = ''
maximum = 0
for game in data:
    if game['stats']['usersrated']:
        if game['stats']['usersrated'] > maximum:
            maximum = game['stats']['usersrated']
            name = game['name']

In [190]:
maximum
name

'Catan'

In [191]:
len(data[25]['image'])

89

In [192]:
Base = declarative_base()

class Boardgame(Base):
    """ Defines the data model for the table `boardgames`. """

    __tablename__ = 'boardgames'

    game_id = Column(String(100), primary_key=True, unique=True, nullable=False)
    name = Column(String(100), unique=False, nullable=False)
    image = Column(String(150), unique=False, nullable=True)
    thumbnail = Column(String(100), unique=False, nullable=True)
    description = Column(Text, unique=False, nullable=True)
    year_published = Column(Date, unique=False, nullable=True)
    min_age = Column(Integer, unique=False, nullable=True)
    number_of_ratings = Column(Integer, unique=False, nullable=True)
    average_user_rating = Column(Float, unique=False, nullable=True)
    number_of_ratings_weight = Column(Integer, unique=False, nullable=True)
    average_user_rating_weight = Column(Float, unique=False, nullable=True)
    bayes_average = Column(Float, unique=False, nullable=True)
                       

    def __repr__(self):
        game_repr = f"<Boardgame(game_id={self.game_id}, name={self.name})>"
        return game_repr

In [193]:
# DataBase User Configurations
DB_HOST = None
DB_PORT = None
DB_USER = None
DB_PW = None

DB_DIALECT = 'mysql+pymysql'
SQLALCHEMY_DATABASE_URI = os.environ.get('SQLALCHEMY_DATABASE_URI')

# Logic for deciding what type of DB to create
if SQLALCHEMY_DATABASE_URI is not None:
    pass
elif DB_HOST is None:
    SQLALCHEMY_DATABASE_URI = 'sqlite:///sample_sqlite_db.db'
else:
    SQLALCHEMY_DATABASE_URI = f'{DB_DIALECT}://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DATABASE}'
            
print(SQLALCHEMY_DATABASE_URI)

sqlite:///sample_sqlite_db.db


In [194]:
# Define Engine String
engine_string = SQLALCHEMY_DATABASE_URI

# Define Engine
engine = create_engine(engine_string)

# Create database
Base.metadata.create_all(engine)

In [195]:
sample_game = data[0]
print(sample_game['id'])
print(sample_game['name'])
print(sample_game['description'])

174430
Gloomhaven
Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives. Players will take on the role of a wandering adventurer with their own special set of skills and their own reasons for traveling to this dark corner of the world. Players must work together out of necessity to clear out menacing dungeons and forgotten ruins. In the process, they will enhance their abilities with experience and loot, discover new locations to explore and plunder, and expand an ever-branching story fueled by the decisions they make.

This is a game with a persistent and changing world that is ideally played over many game sessions. After a scenario, players will make decisions on what to do, which will determine how the story continues, kind of like a “Choose Your Own Adventure” book. Playing through a scenario is a cooperative affair where players will fight against automated monsters using an innovative card system to determine the order of play and what 

In [196]:
data[10]['stats']

{'usersrated': 36337,
 'average': 8.16183,
 'bayesaverage': 8.0103,
 'stddev': 1.46733,
 'median': 0.0,
 'owned': 39761,
 'trading': 563,
 'wanting': 1693,
 'wishing': 11538,
 'numcomments': 5509,
 'numweights': 2475,
 'averageweight': 3.9531,
 'ranks': [{'id': '1',
   'name': 'boardgame',
   'friendlyname': 'Board Game Rank',
   'value': 15},
  {'id': '5497',
   'name': 'strategygames',
   'friendlyname': 'Strategy Game Rank',
   'value': 13}]}

In [197]:
data[10]['stats']['usersrated']

36337

In [198]:
sample_game['year']

2017

In [199]:
# Establish session with database
Session = sessionmaker(bind=engine)
session = Session()

# Define a boardgame in the ORM
game = Boardgame(game_id=sample_game['id'], 
                 name=sample_game['name'], 
                 image=sample_game['image'], 
                 thumbnail = sample_game['thumbnail'], 
                 description=sample_game['description'], 
                 year_published=date(year=sample_game['year'], month=1, day=1 ), 
                 min_age=sample_game['min_age'], 
                 number_of_ratings=sample_game['stats']['usersrated'], 
                 average_user_rating=sample_game['stats']['average'], 
                 number_of_ratings_weight=sample_game['stats']['numweights'], 
                 average_user_rating_weight=sample_game['stats']['averageweight'], 
                 bayes_average=sample_game['stats']['bayesaverage'])

# Add game to session
session.add(game)
# Commit additions to database
session.commit()

In [200]:
# How to check the engine of a given session?
session.connection().engine

Engine(sqlite:///sample_sqlite_db.db)

In [201]:
# Delete all entries in database
session.execute('''DELETE FROM boardgames''')
session.commit()

# Close session
session.close()

In [142]:
# Reading in json file
with open('../../data/games.json', 'r') as json_file:
    games=json.load(json_file)
    print(len(games))
    print(json_file.read())

16311



In [155]:
type(sample_game['id'])

int

In [153]:
type(sample_game['stats']['bayesaverage'])

float

In [138]:
games[0]

{'id': 174430,
 'name': 'Gloomhaven',
 'stats': {'usersrated': 34829,
  'average': 8.83196,
  'bayesaverage': 8.576,
  'stddev': 1.60752,
  'median': 0.0,
  'owned': 55995,
  'trading': 346,
  'wanting': 1417,
  'wishing': 14645,
  'numcomments': 6545,
  'numweights': 1494,
  'averageweight': 3.8253,
  'ranks': [{'id': '1',
    'name': 'boardgame',
    'friendlyname': 'Board Game Rank',
    'value': 1},
   {'id': '5496',
    'name': 'thematic',
    'friendlyname': 'Thematic Rank',
    'value': 1},
   {'id': '5497',
    'name': 'strategygames',
    'friendlyname': 'Strategy Game Rank',
    'value': 1}]},
 'image': 'https://cf.geekdo-images.com/original/img/lDN358RgcYvQfYYN6Oy2TXpifyM=/0x0/pic2437871.jpg',
 'thumbnail': 'https://cf.geekdo-images.com/thumb/img/e7GyV4PaNtwmalU-EQAGecwoBSI=/fit-in/200x150/pic2437871.jpg',
 'artists': ['Alexandr Elichev', 'Josh T. McDowell', 'Alvaro Nebot'],
 'designers': ['Isaac Childres'],
 'year': 2017,
 'description': 'Gloomhaven  is a game of Euro-inspi